In [5]:
from snownlp import SnowNLP
s = SnowNLP(content_list[8])
for sentence in s.sentences:
    print sentence

In [6]:
s1 = SnowNLP(s.sentences[0])
s1.sentiments

0.5

In [1]:
from collections import defaultdict
import os
import re
import codecs
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [57]:
#coding=utf-8
import pandas as pd
import numpy as np
import csv

df1 = pd.read_csv('Kunshan_Case/凤凰网视频.csv')
content_list = list(df1['转发微博内容'])[8000:]

In [58]:
import math
for i in range(len(content_list)):
    if isinstance(content_list[i],float) and math.isnan(float(content_list[i])):
        print i,content_list[i]
        content_list[i] = 0

In [59]:
for i in range(140, len(content_list)):
    content_list[i] = re.sub(r'//.*:','。',content_list[i])
    content_list[i] = re.sub(r'//.*：','。',content_list[i])
    #print i,content_list[i]

In [60]:
import jieba
seg_list = jieba.cut("你今天有点美", cut_all = False)
content_str = "/ ".join(seg_list)
print content_str

In [61]:
result_list = []
for i in range(len(content_list)):
    if isinstance(content_list[i], str):
        seg_list = jieba.cut(content_list[i].decode('utf-8'), cut_all = False)
    content_str = "/ ".join(seg_list)
    print content_str
    temp_list = content_str.split('/')
    result_list.append(temp_list)

In [62]:
#情感词
emotionDict = defaultdict()
emotionList = open('myDict/emotion_dict.txt', 'rw+')
lines = emotionList.readlines()
for item in lines:
    emotionDict[item.split('\t')[0]] = item.split('\t')[1][:-2]
del emotionDict['']
#否定词
notDict = defaultdict()
notList = open('myDict/not_dict.txt', 'rw+')
lines = notList.readlines()
for item in lines:
    notDict[item[:-1]] = -1
#程度副词
degreeDict = defaultdict()
degreeList = open('myDict/degree_dict.txt', 'rw+')
lines = degreeList.readlines()
for item in lines:
    degreeDict[item.split('\t')[0]] = item.split('\t')[1][:-2]
#结尾语气
puncDict = defaultdict()
puncDict['？'] = -1.5
puncDict['！'] = 2
puncDict['~'] = 1.2
puncDict['?'] = -1.5
puncDict['!'] = 2
#停顿
f = open('myDict/stop_dict.txt', 'rw+')
stopList = f.readlines()
for i in range(len(stopList)):
    stopList[i] = stopList[i][:-1]

In [63]:
#计算句子得分
def SentScore(sentence):
    Score = 0
    emotion = 0
    degree = 1
    notword = 0
    punc = 0
    for word in sentence:
        word = word[1:]
        if word not in stopList:
            if word in emotionDict.keys() and word not in notDict.keys() and word not in degreeDict.keys():
                emotion += float(emotionDict[word.encode('utf-8')])
                #print "emotion:",word,float(emotionDict[word.encode('utf-8')])
            elif word in notDict.keys() and word not in degreeDict.keys():
                notword = -1
                #print "not:", word
            elif word in degreeDict.keys():
                degree += float(degreeDict[word.encode('utf-8')])
                #print "degree:", float(degreeDict[word.encode('utf-8')])
            else:
                continue
        elif word in stopList:
            #print "stop"
            if word in puncDict.keys():
                punc += float(puncDict[word.encode('utf-8')])
                #print "punc:", word, float(puncDict[word.encode('utf-8')])
            Score += ((-1)**notword)*degree*emotion+punc
    return Score

In [64]:
SentResult = []
for i in range(len(result_list)):
    result = SentScore(result_list[i])
    SentResult.append(result)
    print i,result
print "Finished"

In [65]:
import xlrd 
import xlwt

f = xlwt.Workbook()
sheet1 = f.add_sheet(u'sheet1',cell_overwrite_ok=True) 
for i in range(len(SentResult)):
    #SentResult[i] = MaxMinNormalization(SentResult[i], max(SentResult), min(SentResult))
    sheet1.write(i,0,SentResult[i])
print "Finished"
f.save('result.xls')

In [18]:
def MaxMinNormalization(x, Max, Min):
    if x>0:
        x = ((x) / Max)*10
    elif x<0:
        x = -(x) / (Min)*10
    elif x==0:
        x = 0
    return x
print SentResult[9]
print MaxMinNormalization(SentResult[9], max(SentResult), min(SentResult))